<a href="https://colab.research.google.com/github/yankeesong/MLE/blob/main/NerfStudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@markdown Install Nerfstudio and Dependencies (~8 min)

# %cd /content/
# !pip install --upgrade pip
# !pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
# %cd /content/
# !gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
# !pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# # Installing COLMAP
# %cd /content/
# !sudo apt-get install \
#     git \
#     cmake \
#     ninja-build \
#     build-essential \
#     libboost-program-options-dev \
#     libboost-filesystem-dev \
#     libboost-graph-dev \
#     libboost-system-dev \
#     libboost-test-dev \
#     libeigen3-dev \
#     libflann-dev \
#     libfreeimage-dev \
#     libmetis-dev \
#     libgoogle-glog-dev \
#     libgflags-dev \
#     libsqlite3-dev \
#     libglew-dev \
#     qtbase5-dev \
#     libqt5opengl5-dev \
#     libcgal-dev \
#     libceres-dev

# !sudo apt-get install gcc-10 g++-10
# !export CC=/usr/bin/gcc-10
# !export CXX=/usr/bin/g++-10
# !export CUDAHOSTCXX=/usr/bin/g++-10

# COLMAP
!export CUDA_PATH=/usr/local/cuda
!export CPLUS_INCLUDE_PATH=$CUDA_PATH/include
!export CMAKE_CXX_FLAGS="-I$CUDA_PATH/include"
!export CMAKE_EXE_LINKER_FLAGS="-L$CUDA_PATH/lib64"
# !git clone https://github.com/colmap/colmap.git
%cd /content/
%cd colmap
!git checkout dev
!rm -rf build
!mkdir build
%cd build
!cmake -DCMAKE_CUDA_ARCHITECTURES='native' .. -GNinja
!ninja
!sudo ninja install

# Install nerfstudio
# %cd /content/
# !pip install git+https://github.com/nerfstudio-project/nerfstudio.git

/content
/content/colmap
M	CMakeLists.txt
Already on 'dev'
Your branch is up to date with 'origin/dev'.
/content/colmap/build
-- The C compiler identification is GNU 11.3.0
-- The CXX compiler identification is GNU 11.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.74.0/BoostConfig.cmake (found version "1.74.0") found components: program_options filesystem graph system 
-- Found FreeImage
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libfreeimage.so
-- Found FLANN
--   Includes : /usr/include
--   Libraries : /usr/lib/x86_64-linux-gnu/libflann.so


In [65]:
cd /usr/local/cuda/include

/usr/local/cuda-11.8/targets/x86_64-linux/include


In [69]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


In [1]:
#@markdown  Downloading and Processing Data
#@markdown Pick the preset scene or upload your own images/video
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDCE4 upload your images' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == 'upload your own Record3D data':
    display(HTML('Zip your Record3D folder, and upload.'))
    display(HTML('More information on Record3D can be found here.'))
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ['upload your images', 'upload your own video']:
    display(HTML('Select your custom data'))
    display(HTML('You can select multiple images by pressing ctrl, cmd or shift and click.'))
    display(HTML('Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content/data/nerfstudio/custom_data/raw_images


Saving 78.jpg to 78.jpg
Saving 77.jpg to 77.jpg
Saving 76.jpg to 76.jpg
Saving 75.jpg to 75.jpg
Saving 74.jpg to 74.jpg
Saving 73.jpg to 73.jpg
Saving 72.jpg to 72.jpg
Saving 71.jpg to 71.jpg
Saving 70.jpg to 70.jpg
Saving 69.jpg to 69.jpg
Saving 68.jpg to 68.jpg
Saving 67.jpg to 67.jpg
Saving 66.jpg to 66.jpg
Saving 65.jpg to 65.jpg
Saving 64.jpg to 64.jpg
Saving 63.jpg to 63.jpg
Saving 62.jpg to 62.jpg
Saving 61.jpg to 61.jpg
Saving 60.jpg to 60.jpg
Saving 59.jpg to 59.jpg
Saving 58.jpg to 58.jpg
Saving 57.jpg to 57.jpg
Saving 56.jpg to 56.jpg
Saving 55.jpg to 55.jpg
Saving 54.jpg to 54.jpg
Saving 53.jpg to 53.jpg
Saving 52.jpg to 52.jpg
Saving 51.jpg to 51.jpg
Saving 50.jpg to 50.jpg
Saving 49.jpg to 49.jpg
Saving 48.jpg to 48.jpg
Saving 47.jpg to 47.jpg
Saving 46.jpg to 46.jpg
Saving 45.jpg to 45.jpg
Saving 44.jpg to 44.jpg
Saving 43.jpg to 43.jpg
Saving 42.jpg to 42.jpg
Saving 41.jpg to 41.jpg
Saving 40.jpg to 40.jpg
Saving 39.jpg to 39.jpg
Saving 38.jpg to 38.jpg
Saving 37.jpg to

In [4]:
!ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/

[06:54:26] 🎉 Done copying images.                                                             ]8;id=120141;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=396610;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/process_data/process_data_utils.py#305\305]8;;\
( ●    ) Copying images...
──────────────────────────────────────────────  💀 💀 💀 ERROR 💀 💀 💀  ───────────────────────────────────────────────
Error running command: colmap -h
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
colmap: error while loading shared libraries: libGLEW.so.2.1: cannot open shared object file: No such file or directory



In [33]:
!grep -r "copying" .

In [29]:

!colmap -h

colmap: symbol lookup error: colmap: undefined symbol: _ZN5ceres6Solver7SummaryC1Ev


In [27]:
!sudo find / -name libceres.so*

/usr/lib/libceres.so.2.0.0
/usr/lib/libceres.so
/usr/lib/libceres.so.2
/usr/lib/libceres.so.1
find: ‘/proc/59/task/59/net’: Invalid argument
find: ‘/proc/59/net’: Invalid argument


In [28]:
!rm /usr/lib/libceres.so.1
!sudo ln -s /usr/lib/libceres.so.2.0.0 /usr/lib/libceres.so.1

In [2]:
#@markdown Set up and Start Viewer

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.427s
https://viewer.nerf.studio/?websocket_url=wss://strong-pens-float.loca.lt
You may need to click Refresh Page after you start training!


In [3]:
#@markdown Start Training

%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    from IPython.core.display import display, HTML
    display(HTML('Error: Data processing did not complete'))
    display(HTML('Please re-run `Downloading and Processing Data`, or view the FAQ for more info.'))

/content
